In [ ]:
import cv2
import numpy as np
import os
import json
from google.colab import drive
import glob

def mount_drive():
    """Google Drive'ı bağlar."""
    drive.mount('/content/drive')
    print("Google Drive bağlandı.")

def get_images_from_folder(folder_path, max_images=3):
    """Takım klasöründen görüntüleri alır."""
    image_extensions = ['*.jpg', '*.jpeg', '*.png']
    image_list = []

    for ext in image_extensions:
        image_list.extend(glob.glob(os.path.join(folder_path, ext)))

    image_list = image_list[:max_images]  # En fazla 3 görüntü
    if len(image_list) < max_images:
        print(f"Uyarı: {folder_path} klasöründe sadece {len(image_list)} görüntü bulundu!")

    images = [cv2.imread(img_path) for img_path in image_list]
    return [img for img in images if img is not None], image_list

def get_rgb_hsv_colors(image):
    """Görüntüden RGB ve HSV ortalamalarını hesaplar."""
    # RGB ortalaması
    pixels = image.reshape(-1, 3)
    rgb_mean = np.mean(pixels, axis=0)  # [B, G, R]

    # HSV ortalaması
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_pixels = hsv_image.reshape(-1, 3)
    hsv_mean = np.mean(hsv_pixels, axis=0)  # [H, S, V]

    return rgb_mean, hsv_mean

def calculate_team_stats(images, team_name, image_paths):
    """Takımın RGB ve HSV ortalamalarını ve varyanslarını hesaplar."""
    rgb_colors = []
    hsv_colors = []
    results = []

    for i, (image, img_path) in enumerate(zip(images, image_paths), 1):
        if image is None:
            continue
        rgb_mean, hsv_mean = get_rgb_hsv_colors(image)
        rgb_colors.append(rgb_mean)
        hsv_colors.append(hsv_mean)
        result = f"{team_name} Forma {i} ({os.path.basename(img_path)}): RGB = {rgb_mean.tolist()}, HSV = {hsv_mean.tolist()}"
        print(result)
        results.append(result)

    if not rgb_colors:
        print(f"Hata: {team_name} için geçerli görüntü bulunamadı!")
        return None, None, None, None, results

    # Ortalamalar
    rgb_team_mean = np.mean(rgb_colors, axis=0)
    hsv_team_mean = np.mean(hsv_colors, axis=0)

    # Varyanslar
    rgb_variance = np.var(rgb_colors, axis=0)
    hsv_variance = np.var(hsv_colors, axis=0)

    results.append(f"\n{team_name} Ortalama RGB: {rgb_team_mean.tolist()}")
    results.append(f"{team_name} Ortalama HSV: {hsv_team_mean.tolist()}")
    results.append(f"{team_name} RGB Varyansı: {rgb_variance.tolist()}")
    results.append(f"{team_name} HSV Varyansı: {hsv_variance.tolist()}")

    print(f"\n{team_name} Ortalama RGB: {rgb_team_mean}")
    print(f"{team_name} Ortalama HSV: {hsv_team_mean}")
    print(f"{team_name} RGB Varyansı: {rgb_variance}")
    print(f"{team_name} HSV Varyansı: {hsv_variance}")

    return rgb_team_mean, hsv_team_mean, rgb_variance, hsv_variance, results

def save_results_to_drive(results, output_folder, txt_filename="results.txt", json_filename="results.json"):
    """Sonuçları metin ve JSON dosyasına kaydedip Drive'a yükler."""
    os.makedirs(output_folder, exist_ok=True)

    # Metin dosyasına kaydet
    txt_path = os.path.join(output_folder, txt_filename)
    with open(txt_path, 'w') as f:
        f.write("\n".join(results))
    print(f"Sonuçlar {txt_path} dosyasına kaydedildi.")

    # JSON dosyasına kaydet
    json_data = {"teams": {}}
    current_team = None
    for line in results:
        if "Sonuçları" in line:
            current_team = line.split("=== ")[1].split(" Sonuçları ===")[0].strip()
            json_data["teams"][current_team] = {"images": [], "averages": {}, "variances": {}}
        elif current_team and "Forma" in line:
            json_data["teams"][current_team]["images"].append(line)
        elif current_team and "Ortalama RGB" in line:
            json_data["teams"][current_team]["averages"]["RGB"] = eval(line.split(": ")[1])
        elif current_team and "Ortalama HSV" in line:
            json_data["teams"][current_team]["averages"]["HSV"] = eval(line.split(": ")[1])
        elif current_team and "RGB Varyansı" in line:
            json_data["teams"][current_team]["variances"]["RGB"] = eval(line.split(": ")[1])
        elif current_team and "HSV Varyansı" in line:
            json_data["teams"][current_team]["variances"]["HSV"] = eval(line.split(": ")[1])
        elif "mesafesi" in line:
            json_data[line.split(": ")[0]] = float(line.split(": ")[1])

    json_path = os.path.join(output_folder, json_filename)
    with open(json_path, 'w') as f:
        json.dump(json_data, f, indent=4)
    print(f"Sonuçlar {json_path} dosyasına kaydedildi.")

# Colab'de OpenCV ve diğer kütüphaneleri yüklemek için gerekliyse
!pip install opencv-python-headless

# Google Drive'ı bağla
mount_drive()

# Ana klasör yolunu belirt
main_folder = "/content/drive/MyDrive/forma"  # Drive'daki klasör yolunu güncelleyin
output_folder = "/content/drive/MyDrive/Forma_Sonuçları"  # Sonuçların kaydedileceği klasör

# Takım klasörlerini al
team_folders = [f for f in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, f))]
if not team_folders:
    print(f"Hata: {main_folder} klasöründe takım klasörü bulunamadı!")
else:
    print(f"Bulunan takım klasörleri: {team_folders}")

# Her takım için hesaplamaları yap
all_results = []
team_stats = {}
for team in team_folders:
    print(f"\n=== {team} Sonuçları ===")
    team_folder = os.path.join(main_folder, team)
    images, image_paths = get_images_from_folder(team_folder)
    rgb_mean, hsv_mean, rgb_var, hsv_var, results = calculate_team_stats(images, team, image_paths)
    all_results.extend(results)
    team_stats[team] = (rgb_mean, hsv_mean, rgb_var, hsv_var)

# Takımlar arası renk farkını hesapla
if len(team_stats) >= 2:
    team_names = list(team_stats.keys())
    rgb_distance = np.sqrt(np.sum((team_stats[team_names[0]][0] - team_stats[team_names[1]][0]) ** 2))
    hsv_distance = np.sqrt(np.sum((team_stats[team_names[0]][1] - team_stats[team_names[1]][1]) ** 2))
    all_results.append(f"\nTakımlar arası RGB mesafesi: {rgb_distance}")
    all_results.append(f"Takımlar arası HSV mesafesi: {hsv_distance}")
    print(f"\nTakımlar arası RGB mesafesi: {rgb_distance}")
    print(f"Takımlar arası HSV mesafesi: {hsv_distance}")

# Sonuçları Drive'a kaydet
save_results_to_drive(all_results, output_folder)

Mounted at /content/drive
Google Drive bağlandı.
Bulunan takım klasörleri: ['beşiktaş', 'fenerbahçe', 'antalyaspor', 'galatasaray']

=== beşiktaş Sonuçları ===
beşiktaş Forma 1 (Ekran görüntüsü 2025-04-23 221329.png): RGB = [61.018233207714474, 57.05782046663711, 61.62752507758812], HSV = [124.95810588007093, 31.316857404123255, 63.37951673686544]
beşiktaş Forma 2 (Ekran görüntüsü 2025-04-23 221337.png): RGB = [101.48130325814536, 96.18202380952381, 100.40043859649123], HSV = [127.27016290726817, 18.23169172932331, 104.01721177944862]
beşiktaş Forma 3 (Ekran görüntüsü 2025-04-23 221807.png): RGB = [37.940678373972, 36.71621215129256, 37.883366121007576], HSV = [70.13866178160401, 25.778129718868254, 39.00302609176939]

beşiktaş Ortalama RGB: [66.81340495 63.31868548 66.63710993]
beşiktaş Ortalama HSV: [107.45564352  25.10889295  68.7999182 ]
beşiktaş RGB Varyansı: [689.69384285 608.96300803 663.94536202]
beşiktaş HSV Varyansı: [697.16949776  28.76086572 7